# Verileri data kısmına input olarak kaggle üzerinden çekip tabloları gösteriyoruz.

In [ ]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib as mpl
import seaborn as sns
import matplotlib.pyplot as plt


train = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv' )
test = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')
submission = pd.read_csv('../input/competitive-data-science-predict-future-sales/sample_submission.csv')
items = pd.read_csv('../input/competitive-data-science-predict-future-sales/items.csv')
item_cats = pd.read_csv('../input/competitive-data-science-predict-future-sales/item_categories.csv')
shops = pd.read_csv('../input/competitive-data-science-predict-future-sales/shops.csv')



print("shops \n ")
print(shops)

print(" \n \n \nitems \n ")
print(items)

print(" \n \n \n item_cats \n ")
print(item_cats)

print(" \n \n \n submission \n ")
print(submission)

print(" \n \n \n train \n ")
print(train)

print(" \n \n \n test \n " )
print(test)


# Train dataların olduğu tablo ile items tablosunu birleştiriyoruz bunun için pandasın merge fonksiyonunu kulanıyoruz.  Amacımız item_id sayısı çok fazla olduğu için bunu 84 tane olan kategori sayısı ile sınırlandırmak . Train tablosu  test tablosuna benzer hale getirip bu 2 verili tablolar üzerinden bir kümeleme yapabilmek. Shop id si üzerinden category id tahmini yaptırmaya çalışıyoruz. Çok fazla kategori olduğu için iyi bir sonuç beklemiyorum. Daha sonraki kısımlarda daha iyi veriler elde edeceğiz.

In [ ]:
traindata = pd.merge(train[['date','shop_id','item_id']], items, how='outer', on='item_id')
traindata=traindata.drop_duplicates(subset=['item_category_id', 'shop_id'], keep='first').sort_values(by=['item_category_id'])
traindata = traindata[['shop_id','item_category_id']].dropna()



testdata = pd.merge(test, items, how='inner', on='item_id')
testdata=testdata.drop_duplicates(subset=['item_category_id', 'shop_id'], keep='first').sort_values(by=['item_category_id'])

testdata = testdata[['shop_id','item_category_id']].dropna()




print(traindata.head(100))

print(testdata)


# Yukarıda bahsettiğimiz sınıflama işleminin gerçekleştirildiği bölüm burasıdır. SVM destek vektör makineleri ile sınıflandırma yapıyoruz ve 0,016 gibi çok kötü bir sonuç elde ediyoruz. Daha sonraki örnekte çok daha yüksek değerlerle daha tahmin edilebilir ve sınıflandırılabilir veriler üzerinden çalışacağız.

In [ ]:
from sklearn import svm


trainx = traindata.iloc[:, :1]
trainy=traindata.iloc[:, 1:2]

testx=testdata.iloc[:, :1]
testy=testdata.iloc[:, 1:2]

clf = svm.SVC()
clf.fit(trainx, trainy)
pred=clf.predict(testx)

#print(testy.transpose())
#print(pred)


from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score


print(" \n\n accuracy score")
print(accuracy_score(testy, pred))
print(" \n\n f1 score")

f1_score(testy, pred, average =None)

# Train ve test diye bize verilen tablolardan anlamlı bir sonuç almanın zorluğu ve özellikle test datasının sadece shop ve item id bilgisini içermesi bu iki tabloyu kullanarak yapabileceğimiz işlemleri kısıtladığı için daha farklı çözüm yollarına gidiyoruz. Ve sadece çok uzun olan train verisini kendi içinde test train diye ayırarak işlem yapmanın daha anlamlı sonuçlar almamızı sağlayacağını düşünerek bu şekilde işlemlere başlıyoruz.  

# Train datamızı detaylı görmek adına okurken date kolonunu tarih olarak alınmasını sağlıyoruz ve daha sonra set index fonksiyonu ile zaman bilgisini index olarak data frame veriyoruz. Veriyi gözlemlemek adına tablonun 1 aylık dönemlerinin ortalamasını alarak 3 milyona yakın veriyi aylık olarak gösteriyoruz.

In [ ]:

mydateparser = lambda x: pd.datetime.strptime(x, "%d.%m.%Y")

train = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv',parse_dates=['date'], date_parser=mydateparser )
test = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')
submission = pd.read_csv('../input/competitive-data-science-predict-future-sales/sample_submission.csv')
items = pd.read_csv('../input/competitive-data-science-predict-future-sales/items.csv')
item_cats = pd.read_csv('../input/competitive-data-science-predict-future-sales/item_categories.csv')
shops = pd.read_csv('../input/competitive-data-science-predict-future-sales/shops.csv')

train=train.set_index('date')
print(train)


montly  =train.resample('1M').mean()

montly

# İlk olarak uyguladığımız dükkan id sine göre kategori tahmini 0 a yakın bir doğruluk oranı verdiği için başka bir deneme yapıyoruz ve bu sefer direk ham veriler üzerinden item id ile shop id yani ürün ile dükkan tahmini yapıyoruz  fakat bunun doğruluğuda 0.0238 çıkıyor ve train - test tablosu üzerinden anlamlı bir şeyler çıkmayacağına karar kılıyoruz.

In [ ]:
train = train[train.item_id<50]
test = test[test.item_id<50]



trainx = train[['item_id','shop_id']].iloc[:, :1]
trainy=train[['item_id','shop_id']].iloc[:, 1:2]

testx=test[['item_id','shop_id']].iloc[:, :1]
testy=test[['item_id','shop_id']].iloc[:, 1:2]

clf = svm.SVC()
clf.fit(trainx, trainy)
pred=clf.predict(testx)

#print(testy.transpose())


from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score


print(" \n\n accuracy score")
print(accuracy_score(testy, pred))

# **Bu kısımda artık train data üzeinden test ve train datamızı oluşturacak ve anlamlı bir sonuç üretmeye çalışacağız. Verimizi tekrar date indexi  ile tarih formatıyla alıyoruz.**

In [ ]:
mydateparser = lambda x: pd.datetime.strptime(x, "%d.%m.%Y")

train = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv',parse_dates=['date'], date_parser=mydateparser )

train=train.set_index('date')
print(train)


# Veriyi daha iyi anlamak için item id ve item fiyatlarını dükkan id si 0 olan dükkan üzerinden çizdirerek  inceliyoruz ve yapacağımız modele karar veriyoruz.

In [ ]:
df=train[train.shop_id==0]
sns.catplot(x="item_id",y="item_price",data=df)

# Dükkanları kendi içerisinde dükkan idlerine göre gruplayıp ortalamalarını alıyoruz. Bu sayede o dükkanda satılan ürünlerin ortalama fiyatı ve ortalama adedi bilgisini elde etmek istiyoruz. Daha sonda bu dükkanlardan ortalama ürün fiyatı 1000 liranın üzerinde olan dükkanlara YüksekKar etiketi ekliyoruz. Eğer ürünlerinin ortalaması 1000 üzerindeyse yüksekkar değeri true  değilse false çeklinde olacak. 

In [ ]:
df2=train.groupby(['shop_id']).mean()
df2['YüksekKar']=df2.item_price > 1000

df2



# Eldeki veriyi 2 kolonlu bir hale getiriyoruz dükkanda satılan ortalama ürün adedi üzerinden bu dükkanın yüksek kar diye tabir ettiğimiz ortalama ürün fiyatı 1000 tl üstünde bir dükkan olup olmadığının tahmin edilmesini istiyoruz. Bunu iki farklı yöntem ile yapacağız önce yukarıda da kullandığımız svm ile daha sonra **KERAS** kullanarak yaptığımız ANN sinir ağları ile modelleyip elde ettiğimiz sonuçlara bakacağız.

In [ ]:
from sklearn import svm


trainx = df2[['item_cnt_day','YüksekKar']].iloc[:49, :1]
trainy = df2[['item_cnt_day','YüksekKar']].iloc[:49, 1:2]



testx = df2[['item_cnt_day','YüksekKar']].iloc[49:, :1]
testy = df2[['item_cnt_day','YüksekKar']].iloc[49:, 1:2]



clf = svm.SVC()
clf.fit(trainx, trainy)
pred=clf.predict(testx)

#print(testy.transpose())
#print(pred)


from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score


print(" \n\n accuracy score")
print(accuracy_score(testy, pred))

# SVM  sınıflandırma yaptığımızda elde ettiğimiz sonuç çok yüksek olmasada ilk başta yaptığımız çok kötü tahminlere göre oldukça iyi düzeyde %81 in üzerinde bir doğruluk oranı ile dükkanların sattığı ortalama ürün üzerinden dükkanların ortalama ürün fiyatının 1000 tl nin üzerinde olup olmadığını tahmin ediyor. 

# Şimdi ise aynı işlemi KERAS üzerinden Neural Network  yapay sinir ağları ile modelleyelim. 1 giriş 1 çıkış ve 2 adet gizli katmanı bulunan bir model tanımlıyoruz. Yukarıda verdiğimiz verinin aynısını veri olarak veriyoruz. 3 vs 2 node olan 2 adet gizli katmanımız 1 adet girdi ve 1 adet çıktı bilgisi içeren düğümümüz yer alıyor. 100 kez epoch ettiriyoruz ve batch_size ımızı 10 olarak ayarlıyoruz. Bu şekilde elde ettiğimiz sonuç %70 lerde kalıyor. 





# Keras ile nn denememiz svm in classification modellemesine göre daha düşük bir doğruluk oranı veriyor. Bu şekilde 2 farklı yöntem ile birden fazla şekilde test işlemi gerçekleştiriyoruz.

In [ ]:

from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense



X = df2[['item_cnt_day','YüksekKar']].iloc[:, :1]
y = df2[['item_cnt_day','YüksekKar']].iloc[:, 1:2]

model = Sequential()
model.add(Dense(3, input_dim=1, activation='relu'))
model.add(Dense(2, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


model.fit(X, y, epochs=100, batch_size=10)

_, accuracy = model.evaluate(X, y)
print('Accuracy: %.2f' % (accuracy*100))

# Sınıflandırma yaparken girdi olarak kullandığımız bir dükkandaki ortalama ürün sayısının şirketlere göre dağılımını görüyoruz buradan dükkanlardaki satılan ürün sayısının genelde 1 ile 1.5 arasına toplandığını görmekteyiz.

In [ ]:
df2[['item_cnt_day','YüksekKar']].plot()

# Projenin amacı kapsamında her dükkan ve her ürün için adet hesaplanması yapılmak istendiğinde train tablosuna join ile yeni veriler eklenir.Ve tablodaki bazı gürültüler temizlenir Örneğin satılan ürün sayısında -1 gibi negatif sayılar tabloda olabiliyor. Bu tarz verilerin temizlenmesi.

In [ ]:
mydateparser = lambda x: pd.datetime.strptime(x, "%d.%m.%Y")

train = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv',parse_dates=['date'], date_parser=mydateparser )   #train tablosu date kolonu tarih formatında olacak şekilde okunur
items = pd.read_csv('../input/competitive-data-science-predict-future-sales/items.csv')

train=train.set_index('date')  #date kolonu tarih formatında okunduktan sonra index yapılır

train= train[train.item_cnt_day>0]  # yanlış girilen gürültülü item bilgileri temizlenir

#traindata = pd.merge(train , items, how='left', on='item_id')    #bu kod ile items tablosu ile train tablosu join yapılır  on kısmında belirtilen parametre üzerinden bu gerçekleştirilir. 
#Bu işlem ek gürültü oluşturduğundan performansı düşürüyor o yüzden kullanılmayıp yorum satırına alınmıştır.

#traindata=traindata.drop_duplicates(subset=['item_category_id', 'shop_id'], keep='first') # tekrar eden veri varsa silinir.
#traindata = traindata[['shop_id','item_category_id']].dropna()





from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense



X = train.iloc[:, :4]    # tablonun ilk 4 kolonu aradığımız sonucun bulunması için verilecek datalar olarak ayarlanır.
y = train.iloc[:, 4:5]  # tablonun 5. kolonu tahmin edilmesi istenen kısımdır. Yani item sayısı

model = Sequential()
model.add(Dense(4, input_dim=4, activation='relu'))   #4 input alan o 4 inputtan 4 düğümlü gizli katmana geçen ilk gizli katman oluşturulur.
model.add(Dense(8, activation='relu'))                #1. gizli katmandan veri alan  8 düğümden oluşan 2. gizli katman eklenir.
model.add(Dense(1, activation='sigmoid'))             # Gizli katmanlardan 1 düğümlü çıkış katmanına gelindiği kısım eklenir. 


model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])   #model için kullanılacak algoritma ve sonuçta çıkarılacak doğruluk metriği belirlenir.


model.fit(X, y, epochs=2, batch_size=100)     #modelin kaç kez eğitileceği kaç epoch dan oluşacağı ve verilerin kaçar kaçar verileceği belirlenir.

_, accuracy = model.evaluate(X, y)
print('Accuracy: %.2f' % (accuracy*100))     # doğruluk değeri bastırılır.



